# Part 3

### Problem Statement
Train a good penalised logistic regression model on the competition data. The
only type of model you should use for this part, is the H2OGeneralizedLinearEstimator
model.

You should:

- Deal appropriately with missings (for all numeric variables, -99 means missing).
- Deal with numerics - i.e. for at least some try linear splines (or another method of your choice to deal with non-linear effects).
- Deal with hccvs (eg using the feature encoding library that we looked at in lecture) (You do not need to deal with low cardinality categorical features since H2O will one-hot them for you).
- Try out some interactions.
- Try out some other features (eg division of numerics).

*Disclaimer: Some preprocessing processes were using the same code as of Part 2

## Set-up

### Import relevant libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

### Set directories and paths

In [2]:
# set directories
print(os.getcwd())
dirRawData = os.path.join('..', 'input')
dirPData   = os.path.join('..', 'PData')
dirPOutput = os.path.join('..', 'POutput')

/home/jovyan/smm284-aml/assignment/jimmy


### Load data

In [3]:
# load data: 250k train data
f_name = '01_df_250k.pickle'
f_path = os.path.join(dirPData, f_name)
PData = pickle.load(open(f_path, "rb"))

# separate data into train/test set
train_set = PData['df_train']
test_set = PData['df_test']

In [4]:
# load variable metadata 
f_name = '01_vars.pickle'
f_path = os.path.join(dirPData, f_name)
var_meta = pickle.load(open(f_path, "rb"))

# extract lists from metadata
var_idx_num = var_meta['vars_ind_numeric']
var_idx_cat = var_meta['vars_ind_categorical']
var_idx_hccv = var_meta['vars_ind_hccv']
var_idx_id = var_meta['vars_notToUse']
var_idx_response = var_meta['var_dep']

### Clean missing values
#### - Deal appropriately with missings (for all numeric variables, -99 means missing).

In [5]:
# replace -99 with np.nan
train_set = train_set.replace(-99, np.nan)
test_set = test_set.replace(-99, np.nan)

In [6]:
# count number of nulls in each column -- train_set
np.array(train_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 102984,
        23788,  23788,  23788,   1852,   2090,  23898,  24321,  11673,
        32806,  33764, 197803,   8895,   6151,  55544,  57869,  57136,
            2,      0,      0,      0,      0,      0,      0,   1092,
            0,      0,      0,      0,      0,      0,      0,      1,
            1,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0])

In [7]:
# count number of nulls in each column -- test_set
np.array(test_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 283601,
            0,      0,      0,      0,      0,      2,      0,      1,
            0,      0,      0,      0,      0,      1,      0,      9,
          226,  10870,      9,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,     17, 131970,  26261,
        26261,  26261,   3429,   3504,  26298,  26356,  18076,   2261,
         2356,   2756,  10130,   6363,  18816,  22520,  20432,      2,
            0,      0,      0,      0,      0,      0,   3560,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

Strategy:
- remove the feature if the number of null values in the `test_set` exceed 250
- drop rows that have leftover null values in the `train_set`
- for leftover null values in the `test_set` impute with `mean` for numerical features and `most frequent` for categorical features

Through data exploration, we decided to remove features that have over 250 null values in the test set. That is because imputation can introduce biases to the data as it is impossible to perfectly predict the true values that are missing. Still, it is not possible to remove observations from the test set concerning the competition. We want to preserve as much information as possible, and thus, for features with fewer than 250 null values (less than 0.1% of the data), imputation for mean / most frequent for numeric / categorical features is incorporated as imputing the small number of observations with these values does not have a significant impact on the overall distribution.

For the train set, after removing features with over 250 null values in the test set, very small numbers of null values are observed (5 in total). We decided to remove those observations, and thus we only lose 3 observations in total.

In [8]:
# number of null values in the test set exceed 250
to_remove = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 250)]

In [9]:
# remove the variables
train_set = train_set.drop(columns=to_remove)
test_set = test_set.drop(columns=to_remove)

In [10]:
# display number of leftover missing values in `train_set`
np.array(train_set.isnull().sum(axis = 0))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
# drop rows that have leftover null values in the `train_set`
train_set = train_set.dropna(axis=0)

In [12]:
# display number of leftover missing values in `test_set`
np.array(test_set.isnull().sum(axis = 0))

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   2,   0,   1,   0,   0,   0,
         0,   0,   1,   0,   9, 226,   9,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,  17,   2,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [13]:
# get columns in the test_set with missing values
impute_col = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 0)]
impute_col

Index(['c09', 'e03', 'e24', 'e17', 'e18', 'e20', 'f10', 'e02'], dtype='object')

In [14]:
# display data for columns that are going to impute
to_impute = test_set[impute_col]
to_impute.head()

,c09,e03,e24,e17,e18,e20,f10,e02
0,B,A,A,920BD,11250,A4893,CKG,6.0
1,B,A,A,F33BC,5614E,D53C5,AIR,15.0
2,B,A,A,F33BC,5614E,D53C5,AEL,15.0
3,A,A,A,861C8,9076B,A4893,CUO,27.0
4,A,A,A,861C8,9076B,A4893,AWV,27.0


In [15]:
# get column indices for numerical and categorical features that need imputation
num_col = to_impute.select_dtypes(include='number').columns
cat_col = to_impute.select_dtypes(include='category').columns
# create imputation pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('cat_impute', SimpleImputer(strategy='most_frequent'), cat_col),
    ('num_impute', SimpleImputer(strategy='mean'), num_col)
])
# impute missing values
imputed_test = pipeline.fit_transform(test_set)

In [16]:
# assign values back to the `test_set`
test_set[impute_col] = imputed_test

Check again the number of missing values

In [17]:
print(f'Number of missing values in `train_set`: {np.sum(np.sum(train_set.isna()))}')
print(f'Number of missing values in `test_set`: {np.sum(np.sum(test_set.isna()))}')

Number of missing values in `train_set`: 0
Number of missing values in `test_set`: 0


As some features were removed, we adjust the metadata

In [18]:
var_idx_num = list(set(var_idx_num) - set(to_remove))
var_idx_cat = list(set(var_idx_cat) - set(to_remove))
var_idx_hccv = list(set(var_idx_hccv) - set(to_remove))

There is also a feature with only 1 unique value: `'e16'`, we removed it from the data.

In [19]:
# column with only 1 unique value
train_set.columns[(train_set.nunique() == 1)]

Index(['e16'], dtype='object')

In [20]:
# remove from the dataframes
train_set = train_set.drop(columns='e16')
test_set = test_set.drop(columns='e16')

# update metadata
var_idx_num.remove('e16')

### Feature engineering

#### - Deal with numerics - i.e. for at least some try linear splines (or another method of your choice to deal with non-linear effects).
  - Apply linear splines to numeric features with more than 20 unique values
  - Spline using the percentiles from the train set and applies the same transformation to the test set. 
  
Similar logic as other preprocessing (e.g. standardisation), the transformation is fitted using the training set and applied to the test. The test set should be kept as unseen (i.e., not knowing the distribution during the training process), and thus, the linear spline should be applied base on the distribution of the train set. 

In [21]:
# extract numeric features from train/test sets
df_train_num = train_set[var_idx_num]
df_test_num = test_set[var_idx_num]

In [22]:
# select only features with over 20 unique values to spline
var_to_spline = list(df_train_num.columns[df_train_num.nunique().values > 20])
print('var_to_spline:\n',
      var_to_spline)

var_to_spline:
 ['e08', 'e05', 'e04', 'e15', 'e02', 'f02']


In [23]:
# define a spline function that transform both train/test set
def fn_spline_train_test(var, x_train, x_test, n_spline):
    # define percentile step size
    step = 100 // n_spline
    # get the percentiles from the train set
    ptiles = np.percentile(x_train, np.arange(step, 100+step, step))
    # initialise dataframes
    train_ptiles = pd.DataFrame({var: x_train})
    test_ptiles = pd.DataFrame({var: x_test})
    # spline the variable
    for idx, ptile in enumerate(ptiles):
        train_ptiles[f'{var}_{str(idx)}'] = np.maximum(0, x_train - ptiles[idx])
        test_ptiles[f'{var}_{str(idx)}'] = np.maximum(0, x_test - ptiles[idx])
    return [train_ptiles, test_ptiles]

In [24]:
# applies linear spline to both train/test sets
for var in var_to_spline:
    # generate a dataframe for spline variables
    train_ptiles, test_ptiles = fn_spline_train_test(var=var, 
                                                     x_train=df_train_num[var],
                                                     x_test=df_test_num[var],
                                                     n_spline=5)
    # drop the variable that were transformed
    df_train_num = df_train_num.drop(columns=[var])
    df_test_num = df_test_num.drop(columns=[var])
    # concat the spline variables
    df_train_num = pd.concat([df_train_num, train_ptiles], axis=1, sort=False)
    df_test_num = pd.concat([df_test_num, test_ptiles], axis=1, sort=False)

In [25]:
# display data
df_test_num.head()

,f26,f25,e06,f01,f08,f22,e07,f15,f13,e09,...,e02_1,e02_2,e02_3,e02_4,f02,f02_0,f02_1,f02_2,f02_3,f02_4
0,0,0,20,33,51,0,0,0,10,43,...,0.0,0.0,0.0,0.0,43,24.0,3.0,0.0,0.0,0.0
1,0,0,76,13,0,0,0,0,9,18,...,0.0,0.0,0.0,0.0,31,12.0,0.0,0.0,0.0,0.0
2,0,0,76,13,51,1,0,0,9,18,...,0.0,0.0,0.0,0.0,64,45.0,24.0,6.0,0.0,0.0
3,0,0,76,13,51,0,0,0,9,18,...,0.0,0.0,0.0,0.0,74,55.0,34.0,16.0,0.0,0.0
4,0,0,76,13,51,0,0,0,8,18,...,0.0,0.0,0.0,0.0,78,59.0,38.0,20.0,0.0,0.0


In [26]:
# create metadata for numeric features after spline
var_idx_spline = df_train_num.columns.tolist()

#### - Deal with hccvs (eg using the feature encoding library that we looked at in lecture) (You do not need to deal with low cardinality categorical features since H2O will one-hot them for you).
For this part, we used Target Encoder to get the mean of the response variable within each class. 

*It performs better than the other encoders.

In [27]:
# extract hccv features from train/test sets
df_train_hccv = train_set[var_idx_hccv]
df_test_hccv = test_set[var_idx_hccv]

In [28]:
# encode hccv with target encoder
enc = TargetEncoder()
# apply target encoder
df_train_hccv = enc.fit_transform(df_train_hccv, train_set[var_idx_response])
df_test_hccv = enc.transform(df_test_hccv)

/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


#### - Try out some interactions.
We decided to try applying interaction among the 6 top categorical variables based on the feature importances performed in part 2. (We used 6 as there is a gap between the 6th and 7th features.

The variables are as follows: `['e21', 'b04', 'f09', 'e20', 'e11', 'a03']`

#### - Try out some other features (e.g. division of numerics).
We decided to try the division of numeric features. Considering the features we used to apply linear spline, they are deemed higher important features with larger varieties (i.e., unique values > 20). 

The variables are as follows: `['e04', 'e05', 'e02', 'e08', 'e15', 'f02']`

From the below table, we noticed that many `e04, e05, etc` have quite close distribution (mean and std). As well as `'f02', 'f01'`;
Therefore, we decided to perform division of these pairs:
`[('f01', 'f02'), ('e04', 'e05'), ('e02', 'e15')]`

In [29]:
# display distribution of numeric features
train_set[var_idx_num].describe().T.sort_values(by='mean', ascending=False).head(15)

,count,mean,std,min,25%,50%,75%,max
e09,249997.0,50.191974,27.948693,18.0,18.0,55.0,76.0,99.0
e08,249997.0,50.031268,28.215267,0.0,16.0,53.0,75.0,99.0
e04,249997.0,49.824678,28.819172,1.0,24.0,49.0,76.0,99.0
f02,249997.0,49.641864,28.872189,2.0,25.0,49.0,74.0,99.0
f01,249997.0,49.588539,28.139675,13.0,13.0,51.0,75.0,99.0
e02,249997.0,49.431249,28.883891,0.0,23.0,50.0,75.0,99.0
e15,249997.0,49.427393,28.604578,0.0,22.0,51.0,74.0,99.0
e06,249997.0,49.387841,27.694698,20.0,20.0,49.0,76.0,99.0
f06,249997.0,49.367396,20.585375,0.0,60.0,60.0,60.0,60.0
f08,249997.0,49.187510,9.329284,0.0,51.0,51.0,51.0,51.0


In [30]:
# division features
# initialise dataframes
df_train_div = pd.DataFrame()
df_test_div = pd.DataFrame()
# specify pairs
div_pairs = [('f01', 'f02'), ('e04', 'e05'), ('e02', 'e15')]
# iterate
for pair in div_pairs:
    # division feature: added by '1e-6' to avoid `0/0` case
    div_train = np.divide((train_set[pair[0]]+1e-6), (train_set[pair[1]]+1e-6))
    div_test = np.divide((test_set[pair[0]]+1e-6), (test_set[pair[1]]+1e-6))
    # name the series
    div_train = div_train.rename(f'{pair[0]}_{pair[1]}')
    div_test = div_test.rename(f'{pair[0]}_{pair[1]}')
    # append data to the initilised dataframes
    df_train_div = pd.concat([df_train_div, pd.DataFrame(div_train)], axis=1, sort=False)
    df_test_div = pd.concat([df_test_div, pd.DataFrame(div_test)], axis=1, sort=False)

# create a metadata for these features
var_idx_div =  df_train_div.columns.tolist()
print(f'Division features: {var_idx_div}')

Division features: ['f01_f02', 'e04_e05', 'e02_e15']


Interaction among categorical pairs causes error in our training which we suspected it was due to some interactions in the val/test sets not exists in the train/design sets. Therefore, for this assignment, we try the interaction term between categorical variables (`['e21', 'b04', 'f09', 'e20', 'e11', 'a03']`) and non-categorical variables that attained the largest normalized coefficients, including target encoded hccv (`'f10'`) and a numeric variable (`'f02'`).

In [31]:
# specify the interaction pairs
for_pairs = ['e21', 'b04', 'f09', 'e20', 'e11', 'a03']
# create interaction pairs
interaction_pairs = []
for i in range(len(for_pairs)):
    for j in range(len(for_pairs)):
        if i < j:
            interaction_pairs.append(tuple([for_pairs[i], for_pairs[j]]))

The below case is not used in our final prediction (This one is for Case 5; see information about Cases below).

In [32]:
# # specify the interaction pairs
# for_pairs_cat = ['e21', 'b04']
# for_pairs_non_cat = ['f10', 'f02']
# # create interaction pairs
# interaction_pairs_addition = []
# for cat in for_pairs_cat:
#     for non_cat in for_pairs_non_cat:
#         interaction_pairs_addition.append(tuple([cat, non_cat]))
# # add additional interaction pairs to the existing set
# interaction_pairs.extend(interaction_pairs_addition)

In [33]:
print(f'Interaction pairs: {interaction_pairs}')

Interaction pairs: [('e21', 'b04'), ('e21', 'f09'), ('e21', 'e20'), ('e21', 'e11'), ('e21', 'a03'), ('b04', 'f09'), ('b04', 'e20'), ('b04', 'e11'), ('b04', 'a03'), ('f09', 'e20'), ('f09', 'e11'), ('f09', 'a03'), ('e20', 'e11'), ('e20', 'a03'), ('e11', 'a03')]


#### Concatenate all the preprocessed dataframes
Prepare dataframes for model training: train/val/test/design sets

In [34]:
# exclude hccv from cat variables metadata
var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# components = ['numeric and splines', 'categorical', 'hccv', 'division']
df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], df_train_hccv,
                           df_train_div, train_set[var_idx_response]], axis=1, sort=False)
df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], df_test_hccv,
                        df_test_div], axis=1, sort=False)

# split design into train/val sets
df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

Alternative settings: This is used to explore the result. The above cell contains the best alternative from this experiment.

In [35]:
# """
# Case 1: No transformation
# """
# # exclude hccv from cat variables metadata
# var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# # components = ['numeric and splines', 'categorical', 'hccv', 'division']
# df_design_all = pd.concat([train_set[var_idx_num], train_set[var_idx_cat_not_hccv], df_train_hccv,
#                            train_set[var_idx_response]], axis=1, sort=False)
# df_test_all = pd.concat([test_set[var_idx_num], test_set[var_idx_cat_not_hccv], df_test_hccv],
#                          axis=1, sort=False)

# # split design into train/val sets
# df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

In [36]:
# """
# Case 2: Add spline
# """
# # exclude hccv from cat variables metadata
# var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# # components = ['numeric and splines', 'categorical', 'hccv', 'division']
# df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], df_train_hccv,
#                            train_set[var_idx_response]], axis=1, sort=False)
# df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], df_test_hccv],
#                         axis=1, sort=False)

# # split design into train/val sets
# df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

Case 3, 4, 5 literally use the same setting at this stage

In [37]:
# """
# Case 3: Add division features
# """
# # exclude hccv from cat variables metadata
# var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# # components = ['numeric and splines', 'categorical', 'hccv', 'division']
# df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], df_train_hccv,
#                            df_train_div, train_set[var_idx_response]], axis=1, sort=False)
# df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], df_test_hccv,
#                         df_test_div], axis=1, sort=False)

# # split design into train/val sets
# df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

Case 4 was used for our final prediction for Part 3

In [38]:
"""
Case 4: Add interaction terms (categorical variables)
-- based on variable importances with tree-based selection in Part 2.
"""
# exclude hccv from cat variables metadata
var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# components = ['numeric and splines', 'categorical', 'hccv', 'division']
df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], df_train_hccv,
                           df_train_div, train_set[var_idx_response]], axis=1, sort=False)
df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], df_test_hccv,
                        df_test_div], axis=1, sort=False)

# split design into train/val sets
df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

In [39]:
# """
# Case 5: Add interaction terms (categorical-hccv/numeric)
# -- based on normalised coefficient of the previous model above.
# """
# # exclude hccv from cat variables metadata
# var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# # components = ['numeric and splines', 'categorical', 'hccv', 'division']
# df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], df_train_hccv,
#                            df_train_div, train_set[var_idx_response]], axis=1, sort=False)
# df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], df_test_hccv,
#                         df_test_div], axis=1, sort=False)

# # split design into train/val sets
# df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

Model evaluation log for the five experimental cases:

|        |    Explanation    |  Best Alpha   |      Best Lambda      |     Train AUC      |       Val AUC       |
|-------:|:------------------|:--------------|:----------------------|:-------------------|---------------------|
| Case 1 | No transformation |      1.0      |         7e-05         |      0.84513       |       0.84118       |
| Case 2 |    Add spline     |      1.0      |         7e-05         |      0.84631       |       0.84246       |
| Case 3 |   Add division    |      1.0      |         7e-05         |      0.84633       |       0.84248       |
| Case 4 |  Add interaction  |      0.0      |         0.0010        |      0.85023       |       0.84482       |
| Case 5 |  More interaction |      0.0      |         0.0016        |      0.84963       |       0.84448       |

Best Model: Case 4 - Spline + Division + Categorical interaction
- Alpha: 0.0 -- Ridge regression
- Lambda: 0.001

From the table, we observed that there is a significant improvement when we incorporate linear spline and adding categorical interaction terms. The improvement of results applies to both the training and the validation sets. 

Adding division terms does not add much improvement to the results. However, the interaction term with target encoded and numeric variables worsen the performance. This could be that those interaction terms become noise to the model.

The first three cases used Lasso regression, while Case 4 and Case 5 used Ridge regression. None of the best models came from Elastic Net.

#### Standardise numeric features

- Standardise train/val set for model selection

In [40]:
# get column indices for numerical features that need to standardise
# -- index intersection/union are used so we can apply the same function on all the Cases above
all_num_col = train_set[var_idx_num].columns.union(var_idx_spline).union(var_idx_div)
num_col = df_train_all.columns.intersection(all_num_col)

# create standardise pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('standardise', StandardScaler(), num_col)
])

# fit standardisation on the train set
train_scale = pipeline.fit_transform(df_train_all)
# apply the standardisation to the val set
val_scale = pipeline.transform(df_val_all)

# replace the values with scaled values
df_train_all[num_col] = train_scale
df_val_all[num_col] = val_scale

<ipython-input-40-5622a1ff2df5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_all[num_col] = train_scale
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<ipython-input-40-5622a1ff2df5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

- Standardise design/test set for test set prediction

In [41]:
# get column indices for numerical features that need to standardise
# -- index intersection/union are used so we can apply the same function on all the Cases above
all_num_col = train_set[var_idx_num].columns.union(var_idx_spline).union(var_idx_div)
num_col = df_design_all.columns.intersection(all_num_col)

# create standardise pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('standardise', StandardScaler(), num_col)
])

# fit standardisation on the design set
# -- need to remove the response during transform so the column index match with the test set
design_scale = pipeline.fit_transform(df_design_all.iloc[:,:-1])
# apply the standardisation to the test set
test_scale = pipeline.transform(df_test_all)

# replace the values with scaled values
df_design_all[num_col] = design_scale
df_test_all[num_col] = test_scale

#### Start H2O

In [42]:
# connect to h2o via localhost
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_292"; OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~20.04-b10); OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5664aubp
  JVM stdout: /tmp/tmp5664aubp/h2o_jovyan_started_from_python.out
  JVM stderr: /tmp/tmp5664aubp/h2o_jovyan_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,2 months and 15 days
H2O_cluster_name:,H2O_from_python_jovyan_ib47j5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.971 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [43]:
# load data to h2o java vm
h2o_df_train = h2o.H2OFrame(df_train_all, destination_frame='df_train')
h2o_df_val = h2o.H2OFrame(df_val_all, destination_frame='df_val')
h2o_df_design = h2o.H2OFrame(df_design_all, destination_frame='df_design')
h2o_df_test = h2o.H2OFrame(df_test_all, destination_frame='df_test')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


#### Model training
- Penalised logistic regression with H2OGeneralizeddLinearEstimator()

Use the below cell when training the model without interaction terms.

In [44]:
%%time
"""Without Interaction Term"""
# specify dependent/independent variables
y = var_idx_response[0]
X = list(set(h2o_df_train.columns) - set(var_idx_response))

# initialise the tuning grid for alpha
# -- the distribution between L1-norm and L2 norm: 
# -- alpha = 0 corresponds to Ridge Regression
# -- alpha = 1 corresponds to Lasso Regression
alpha_grids = np.linspace(0, 1, 4).tolist()

hyper_parameters = {'alpha': alpha_grids,
                   }

# setup search criteria
criteria = {'strategy': 'RandomDiscrete',
            'stopping_metric' : 'auc',
            'stopping_rounds': 3,
            'seed': 999
           }

# specify the model
grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family='binomial',
                                                   nfolds=5,
                                                   lambda_search=True,
                                                   standardize=False,
                                                   seed=999),
                     hyper_params=hyper_parameters,
                     grid_id='g1',
                     search_criteria=criteria)

# train the model
grid.train(y=y,
           x=X,
           training_frame=h2o_df_train,
           validation_frame=h2o_df_val)

# examine the grid performance
grid = grid.get_grid(sort_by='auc', decreasing=True)
print(grid)

# get the best model
best_model = grid.models[0]

# print results from the best model
print(f'Best Alpha: {H2OGeneralizedLinearEstimator.getAlphaBest(best_model)}')
print(f'Best Lambda: {H2OGeneralizedLinearEstimator.getLambdaBest(best_model)}')
print(f'Train AUC: {best_model.auc(valid=False)}')
print(f'Val AUC: {best_model.auc(valid=True)}')

# this only show one of the results we tried during the experiment
# the result below is corresponding to Case 3.
# -- even with seed set, the psuedorandomness in the model training
# -- does not give the completely same results concerning very small digits

# expected results:
# Best Alpha: 1.0
# Best Lambda: 7.068312175009902e-05
# Train AUC: 0.8463336124884145
# Val AUC: 0.842472021137556

glm Grid Build progress: |████████████████████████████████████████████████| 100%
                    alpha   model_ids                 auc
0                   [1.0]  g1_model_2  0.8457717321312107
1    [0.3333333333333333]  g1_model_3   0.845745411029609
2    [0.6666666666666666]  g1_model_1  0.8457402048062587
3                   [0.0]  g1_model_4  0.8457039653025147

Best Alpha: 1.0
Best Lambda: 7.068312175009902e-05
Train AUC: 0.8463336124884145
Val AUC: 0.842472021137556
CPU times: user 3.93 s, sys: 316 ms, total: 4.25 s
Wall time: 25min 30s


In [45]:
# print normalised coefficients for each features (sorted descending)
# -- this is use when identifying which variables to try for the interaction pairs
for key in sorted(best_model.coef_norm(), key=best_model.coef_norm().get, reverse=True):
    print(key, best_model.coef_norm()[key])

f10 1.5837335055382613
f03.F 1.1026257955021492
f03.E 0.7491052315925488
f02 0.383319781381615
f09.G 0.33031447739212466
f27.F 0.2884689894949824
f03.D 0.2832475905124322
e11.B 0.21759193806147825
e18 0.21407681897055295
e14.F 0.17863368581660063
f04.A 0.15582334410115833
e11.C 0.1485170729105255
e14.D 0.12783801719986906
f05.D 0.09491096440135381
e20.B2D1C 0.0909915310463764
f27.B 0.0864175216184668
e15_0 0.07830260721736312
e03.C 0.07592116543074066
e08_3 0.0744209242821456
f29.E 0.07132817396952214
e20.A5DA8 0.06958239815848948
e21.Q 0.06877183915474061
f06 0.06767661284354115
e24.Q 0.06607980058250987
e03.D 0.06234158646287696
a02.B 0.06055649739524875
e21.C 0.060470302862944164
e11.E 0.04187167598784827
a03.C 0.04085814140568678
c08.B 0.0404900869410792
e21.G 0.0400222491752231
e21.H 0.03634934664679317
a03.L 0.03440098107295733
e21.A 0.033275264076709624
b02.Y 0.03278288691377278
f08 0.032243216418911556
e05_3 0.03204430400933947
e05_0 0.030211650740889114
e13.F 0.029624367694978

For the case with interaction terms, there seems to be a conflict between cross-validation(`nfolds`) and `interaction_pairs` arguments which causes an error. Also, `H2OGridSearch()` function does not work properly. Therefore, we decided to sacrifice the benefit from cross-validation to appreciate the interaction_pairs built-in functionality from `H2O`. For grid search, we decided to use for loop to perform it manually. 

Disclaimer: Though we have heard from the email later regarding your suggestion to use `interaction()` function to manually create the interaction terms before feeding into the model training function, we decided to propose our solution to address the issue differently as it, more or less, seems to work fine for this data set (i.e., the model with interaction terms performed best as per our experiment).


In [46]:
%%time
"""With Interaction Terms"""
# specify dependent/independent variables
y = var_idx_response[0]
X = list(set(h2o_df_train.columns) - set(var_idx_response))

# initialise the tuning grid for alpha
# -- the distribution between L1-norm and L2 norm: 
# -- alpha = 0 corresponds to Ridge Regression
# -- alpha = 1 corresponds to Lasso Regression
alpha_grids = np.linspace(0, 1, 4).tolist()

for alpha_ in alpha_grids:
    # specify the model
    model = H2OGeneralizedLinearEstimator(alpha=alpha_,
                                          family='binomial',
                                          lambda_search=True,
                                          lambda_min_ratio=1e-6,
                                          early_stopping=False,
                                          standardize=False,
                                          interaction_pairs=interaction_pairs,
                                          seed=999)

    # train the model
    model.train(y=y,
                x=X,
                training_frame=h2o_df_train,
                validation_frame=h2o_df_val)
    # print the result
    print(f'Alpha: {H2OGeneralizedLinearEstimator.getAlphaBest(model)}')
    print(f'Best Lambda: {H2OGeneralizedLinearEstimator.getLambdaBest(model)}')
    print(f'Train AUC: {model.auc(valid=False)}')
    print(f'Val AUC: {model.auc(valid=True)}')
    print('-------------------------------------')
    
# this only show one of the results we tried during the experiment
# the result below is corresponding to Case 4 -- the optimal model: Alpha = 0.0 - Ridge regression.
# -- even with seed set, the psuedorandomness in the model training
# -- does not give the completely same results concerning very small digits

# expected results:
# Alpha: 0.0
# Best Lambda: 0.0009749633940760714
# Train AUC: 0.8502312086820765
# Val AUC: 0.8448210360918208

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [e15_4, e02_4, e05_4, e04_4, f02_4, e08_4]
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
Alpha: 0.0
Best Lambda: 0.0009749633940760714
Train AUC: 0.8502375475868515
Val AUC: 0.8448112578106212
-------------------------------------
glm Model Build progress: |███████████████████████████████████████████████| 100%
Alpha: 0.3333333333333333
Best Lambda: 0.0009395104677450065
Train AUC: 0.8477123725660951
Val AUC: 0.8437121988532907
-------------------------------------
glm Model Build progress: |███████████████████████████████████████████████| 100%
Alpha: 0.6666666666666666
Best Lambda: 0.0009438477190242957
Train AUC: 0.8465756363050191
Val AUC: 0.8425353122371827
-------------------------------------
glm Model Build progress: |███████████████████████████████████████████████| 100%
Alpha: 1.0
Best Lambda: 0.00072346367835892
Train AUC: 0.8465686025969433
Val AUC: 0.8425245855019254
-------------------------------------
CPU times: user 3.5 s, sys: 533 ms, total: 4.04 s
Wall time: 8min 18s


#### Make prediction on the test set

In [47]:
%%time
# specify dependent/independent variables
y = var_idx_response[0]
X = list(set(h2o_df_design.columns) - set(var_idx_response))

# specify the parameter tuned through experiments
best_alpha = 0
best_lambda = 0.001

# train the best model we got with the design set (train+val)
model = H2OGeneralizedLinearEstimator(alpha=best_alpha,
                                      lambda_=best_lambda,
                                      family='binomial',
                                      lambda_search=False,
                                      standardize=False,
                                      seed=999)
# train the model
model.train(y=y,
            x=X,
            training_frame=h2o_df_design)

glm Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 54.2 ms, sys: 8.61 ms, total: 62.9 ms
Wall time: 3.77 s


In [48]:
# print AUC score on the design set
# -- Design AUC: 0.8446103793960728
print(f'Design AUC: {model.auc(valid=False)}')

Design AUC: 0.8450130778543783


Some classes in the test categorical features are not in the data used for training. H2O automatically deal with these unseen class by "Skip" method. 

(ref: https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html)

In [49]:
# predict the model on design and test sets
model_pred_design = model.predict(h2o_df_design[h2o_df_test.columns])
model_pred_test   = model.predict(h2o_df_test)

# get the probability of the prediction of being '1'
model_pred_test = model_pred_test.as_data_frame()['p1']

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: ["30146", "BE271"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: ["Q", "S"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: ["J", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: ["J", "M", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'a18' has levels not trained on: ["D"]
  warnings.warn(w)


#### Create submission file

In [50]:
submission = pd.DataFrame({
    'unique_id': test_set['unique_id'],
    'Predicted': model_pred_test
})
# save the submission file
f_name = 'Group_8_part3.csv'
f_path = os.path.join(dirPOutput, f_name)
submission.to_csv(f_path, index=False)

In [51]:
# preview the submission file
submission

,unique_id,Predicted
0,6,0.084786
1,16,0.915669
2,17,0.218722
3,18,0.565979
4,19,0.099945
...,...,...
296685,2265630,0.918705
296686,2265631,0.097037
296687,2265632,0.590760
296688,2265637,0.178242


In [52]:
# shutdown h2o cluster
h2o.cluster().shutdown()

H2O session _sid_9ffe closed.


### Note on Part 3

#### Kaggle score: 0.84734
<img src="../screenshots/Group_8_part3_screenshot.png">

For Part 3, the logical processes for our solution are as follows:
- Strategically cleaned the missing values
- Applied linear spline (n=5; equidistant percentile between 0,1) to numeric features that have more than 20 unique values
- Applied target encoder to the high cardinality categorical variables
- Created division terms among numeric variables
- Created interaction terms among the most important categorical variables
- Created interaction terms between categorical variables and hccv/numeric variables
- Tuned the elastic net model with different combinations of features
- Fitted the final model on the design set: model = Ridge regression with lambda = 0.001
- Predict the test set and create the submission file for the Kaggle competition

Compare the result to Part 2:

| Kaggle Competition   |        Part 2          |        Part 3         |
|:---------------------|:-----------------------|:----------------------|
|         AUC          |        0.62644         |       0.84734         |

From the 5 Cases table above, we noticed that including hccv variables in the training set significantly boosted the model performance, as we can see a large improvement from Part 2. Engineered features such as interaction terms, even though we do not know what each variable represented, can also improve the performance of the model.